In [3]:
import requests
import pandas as pd
import numpy as np
import json

In [6]:
embedding_api = "http://localhost:11434/api/embeddings"


def get_embedding(text):
    data = {"model": "nomic-embed-text", "prompt": text}

    response = requests.post(embedding_api, json=data)
    return np.array(response.json()['embedding'])

In [7]:
names = []

with open('./terraformToAws.json') as file:
    data1 = json.load(file)
    names.extend(data1.keys())

with open('./resourceIdentifiers.json') as file:
    data2 = json.load(file)
    names.extend(data2.keys())

In [8]:
embeddings = { name: get_embedding(name) for name in names}

In [12]:
# create a dataframe
df = pd.DataFrame(embeddings)

In [13]:
def cosine_similarify(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

In [14]:
df2 = df.filter(like='::')

def most_similar_words(word, n=5):
    similarities = df2.apply(lambda x: cosine_similarify(df[word], x), axis=0)
    return similarities.sort_values(ascending=False).head(n)

In [15]:
most_similar = pd.Series({name: most_similar_words(name, n=1).index[0] for name in data1})

In [17]:
most_similar

aws_eks_node_group                                         aws::eks::nodegroup
aws_eks_identity_provider_config              aws::eks::identityproviderconfig
aws_eks_cluster                                              aws::eks::cluster
aws_eks_access_policy_association                   aws::organizations::policy
aws_eks_access_entry                                           aws::eks::addon
                                                            ...               
aws_iam_access_key                                         aws::iam::accesskey
aws_iam_group_membership                                       aws::iam::group
aws_iam_security_token_service_preferences                    aws::iam::policy
aws_iam_saml_provider                                   aws::iam::samlprovider
aws_iam_user                                                    aws::iam::user
Length: 1246, dtype: object